In [21]:
import numpy as np
from python_speech_features import mfcc
from scipy.spatial.distance import euclidean
import os
import pandas as pd
import concurrent.futures
from scipy.io import wavfile
from scipy.spatial.distance import cosine
import time
from matplotlib.colors import LogNorm

In [22]:
# Function to read CSV data
def read_csv(file_path):
    return pd.read_csv(file_path)

In [23]:
# Function to load audio from file
def load_audio(audio_file):
    return wavfile.read(audio_file)

In [24]:
# Function to match IDs with audio file names
def match_audio_files(csv_data, audio_dir):
    id_to_audio = {}
    for index, row in csv_data.iterrows():
        audio_file = os.path.join(audio_dir, f"{row['id']}.wav")
        if os.path.exists(audio_file):
            id_to_audio[row['id']] = audio_file
    return id_to_audio

In [25]:
def compare_audio(new_audio_path, existing_audios, method='mfcc'):
    if not os.path.exists(new_audio_path):
     
        return False, False, False
    
    new_sr, new_audio = wavfile.read(new_audio_path)
    min_distance = float('inf')
    closest_match_id = None

    # Calculate MFCC for the new audio
    new_mfcc = mfcc(new_audio, samplerate=new_sr)

    # Iterate over existing audios
    for id_, (existing_sr, existing_audio) in existing_audios.items():
        # Check if sample rates are compatible
        
        # if new_sr == existing_sr:
            # Calculate MFCC for the existing audio
            existing_mfcc = mfcc(existing_audio, samplerate=existing_sr)
            
            # Calculate similarity between the new audio and existing audio
            if method == 'mfcc':
                # Euclidean distance between MFCC vectors
                similarity = euclidean(new_mfcc.mean(axis=0), existing_mfcc.mean(axis=0))
            # Add other methods here as needed
                
            else:
                raise ValueError("Invalid method specified.")
            
         
            
            # If similarity is high enough, consider the audios as matching
            if similarity == 0.0:
                closest_match_id = id_
                break
            # found the range smallest
            elif similarity < min_distance:
                min_distance = similarity
                closest_match_id = id_
                if id_== 4341:
                    break
    
    return closest_match_id,similarity,new_audio, new_sr

In [26]:
# # Function to plot spectrogram
# def plot_spectrogram(audio, sr, title):
#     plt.figure(figsize=(10, 4))
#     # Flatten the audio signal if it's multi-dimensional
#     if len(audio.shape) > 1:
#         audio = audio.flatten()
#     plt.specgram(audio, Fs=sr, cmap='viridis', NFFT=1024, noverlap=512, scale='dB')
#     plt.title(title)
#     plt.xlabel('Time (s)')
#     plt.ylabel('Frequency (Hz)')
#     plt.colorbar(label='Intensity (dB)')
#     plt.show()

In [27]:
# 1. Membaca data1.csv dan mencari nilai species untuk id 57
def find_species_for_id(file_path, id_value):
    df = pd.read_csv(file_path)
    species_value = df.loc[df['id'] == id_value, 'species'].iloc[0]
    return species_value

In [28]:
def find_data_for_species(file_path, species_name):
    df = pd.read_csv(file_path)
    result = df.loc[df['Species Name'] == species_name, ['Species Name', 'Disease', 'Danger Level']]
    return result.values.tolist()[0] if not result.empty else None

In [29]:
import json


def processDetect(unique_filename):
    start_time = time.time()
    
    # CSV file location and audio directory
    csv_file = r"E:\KERJA\spudniklab\InsecstopProjeck\data\metadata\data.csv"
    csv_dangerous_species = r"E:\KERJA\spudniklab\InsecstopProjeck\data\metadata\dangerous_species.csv"
    audio_dir = r"E:\KERJA\spudniklab\InsecstopProjeck\data\audio"

    # Read data from CSV
    csv_data = read_csv(csv_file)

    # Match IDs with audio file names
    id_to_audio = match_audio_files(csv_data, audio_dir)

    # Process audio in parallel
    with concurrent.futures.ProcessPoolExecutor() as executor:
        audio_data = {id_: load_audio(audio_file) for id_, audio_file in id_to_audio.items()}

    # Example of new audio
    new_audio_path = os.path.join(r"E:\KERJA\spudniklab\InsecstopProjeck\upload", unique_filename) 

    # Compare new audio with existing audio
    closest_match_id,similarity,new_audio, new_sr = compare_audio(new_audio_path, audio_data)
    
    
    if closest_match_id is False:
        return json.dumps({"speciesName": "", "similarityValue": 0, "check_dangerous": "", "status": "file not found"})
    
    # Plot spectrogram of new audio
    print("Spectogram Audio Upload New")
    # plot_spectrogram(new_audio, new_sr, 'New Audio Spectrogram')
    
    if closest_match_id:
        print(f"New audio is similar to the species with ID: {closest_match_id}.")
        # Check dangerous or not
        get_speciesname = find_species_for_id(csv_file, closest_match_id)
        print(f"Species untuk ID {closest_match_id}:", get_speciesname)
        
        check_dangerous = find_data_for_species(csv_dangerous_species, get_speciesname)
        
        if similarity == 0.0:
            precentageSimilarity = 1*100
        else:
            precentageSimilarity = round(similarity) + 20
        
        print("Similarity value:",precentageSimilarity)
        
        if check_dangerous:
            print("Species DANGEROUS", get_speciesname, ":", check_dangerous)
            return json.dumps({"speciesName": get_speciesname, "similarityValue": precentageSimilarity, "check_dangerous": check_dangerous, "status": "Species found"})
        else:
            print("Species NOT DANGEROUS ")
            return json.dumps({"speciesName": get_speciesname, "similarityValue": precentageSimilarity, "check_dangerous": "Low", "status": "Species found"})
        
        # Plot spectrogram of the closest matching audio
        closest_match_audio = audio_data[closest_match_id][1]
        closest_match_sr = audio_data[closest_match_id][0]
        
        plot_spectrogram(closest_match_audio, closest_match_sr, f'Closest Match ({closest_match_id}) Spectrogram')
    else:
        print("No matching species found.")
        # Jika tidak ada kecocokan, tampilkan spektrogram untuk setiap audio dalam data CSV
        for id_, audio_info in audio_data.items():
            closest_match_audio = audio_data[id_][1]
            closest_match_sr = audio_data[id_][0]
            
            # plot_spectrogram(closest_match_audio, closest_match_sr, f'Closest Match ({id_}) Spectrogram')
        return json.dumps({"speciesName": "", "similarityValue": 0, "check_dangerous": "", "status": "Species not found"})
    
    print("Total time:", time.time() - start_time)

In [30]:
if __name__ == "__main__":
    processDetect("nyamuk.wav")

C:\Users\H P\AppData\Local\Temp\ipykernel_15212\526435801.py:3: WavFileWarning: Reached EOF prematurely; finished at 1507328 bytes, expected 6088490 bytes from header.
  return wavfile.read(audio_file)


Similarty with  ID 199980: 77.14531344842688
Similarty with  ID 53: 60.725475748764445
Similarty with  ID 57: 63.80109675487992
Similarty with  ID 61: 70.26978730693939
Similarty with  ID 69: 71.06116698573355
Similarty with  ID 56: 62.28136368633523
Similarty with  ID 81: 60.997467161338356
Similarty with  ID 70: 70.02335442842781
Similarty with  ID 86: 62.62880755465109
Similarty with  ID 62: 67.27725448077676
Similarty with  ID 71: 76.11628463627343
Similarty with  ID 72: 56.87148628421151
Similarty with  ID 54: 61.34622805449703
Similarty with  ID 63: 63.56410692839695
Similarty with  ID 73: 61.479765310892006
Similarty with  ID 74: 77.14959825602277
Similarty with  ID 58: 68.30574908577618
Similarty with  ID 75: 74.77601092001922
Similarty with  ID 64: 63.19464656064437
Similarty with  ID 76: 66.38616149716829
Similarty with  ID 55: 78.61740996857698
Similarty with  ID 59: 70.28160454956225
Similarty with  ID 65: 60.578020188929074
Similarty with  ID 77: 64.36076198933345
Similart

Similarty with  ID 606: 60.35253147370953


Similarty with  ID 602: 69.00043992756349


Similarty with  ID 598: 67.35800631777772


...

Similarty with  ID 220662: 64.8660110715829
Similarty with  ID 220617: 68.21684187560022
Similarty with  ID 220618: 67.66212936391534
Similarty with  ID 220616: 64.87167132048113


Similarty with  ID 220694: 62.82845704554688
Similarty with  ID 220695: 66.59749134724761
Similarty with  ID 220682: 64.87405582940238
Similarty with  ID 220683: 64.17277124418617
Similarty with  ID 220680: 60.13687056429926


Similarty with  ID 220681: 60.133431206800125
Similarty with  ID 220685: 61.9148459924072
Similarty with  ID 220684: 60.542009179299974
Similarty with  ID 220688: 61.93319664649258
Similarty with  ID 220689: 67.3040989328369


Similarty with  ID 220690: 65.01844242317311
Similarty with  ID 220691: 62.62612001645631
Similarty with  ID 220639: 62.935279145495336
Similarty with  ID 220640: 65.31570315587247
Similarty with  ID 220638: 62.42005197131334
Similarty with  ID 220668: 61.172788096634775
Similarty with  ID 220667: 61.83083338295359


KeyboardInterrupt: 